<h1>Tutorials for RoboManipBaselines</h1>

このシートは
<a href="https://sites.google.com/view/sice-si-manipulation/Wakate" target="_blank">
  マニピュレーション若手の会
</a>
がワークショップのために作成した非公式なものです。<br>
<strong>開発者および公式リポジトリに本スクリプトについての問い合わせはしないでください。</strong>

<h2>ワークショップ概要</h2>

<p>
  RoboManipBaselinesは、産業技術総合研究所の室岡氏らによって開発された、最先端の模倣学習手法を実世界およびシミュレーション環境で再現・検証するためのソフトウェアフレームワークです。本来は、外部グラフィックカードなどを備えた一定の性能を持つ計算機を用いて実行することが推奨されますが、ワークショップではラップトップパソコンなどの性能に依存せずに試せるよう、Google Colab向けのPythonノートブックを提供しています。
</p>

<p>
  <img src="https://www.aist.go.jp/Portals/0/resource_images/aist_j/press_release/pr2025/pr20250123_2/fig2.jpg" alt="RoboManipBaselines overview">
</p>

<p>
  （参照元：
  <a href="https://github.com/isri-aist/RoboManipBaselines" target="_blank">
    https://github.com/isri-aist/RoboManipBaselines
  </a>
  ）
</p>

<p>
  [
  <a href="https://github.com/isri-aist/RoboManipBaselines.git" target="_blank">
    GitHub Repository
  </a>
  ] [
  <a href="https://arxiv.org/abs/2509.17057" target="_blank">
    arXiv
  </a>
  ] [
  <a href="https://isri-aist.github.io/RoboManipBaselines-ProjectPage/" target="_blank">
    Project Page
  </a>
  ]
</p>


<h2>Google Colabの設定</h2>

* runtime: Python3
* Hardware Accelerator: T4 GPU
* Version: 2025.10 or latest

## Install RoboManipBaselines

This section guides you through setting up the necessary environment and installing the RoboManipBaselines library from its GitHub repository, including its submodules and specific dependencies like ACT (Action Chunking with Transformer).

### Clone and Install RoboManipBaselines

Next, we'll clone the `RoboManipBaselines` GitHub repository along with all its submodules. This command performs a recursive clone to get all necessary components. After cloning, we navigate into the repository and install it in "editable" mode within our virtual environment. The `-e .` flag allows for local code changes to be reflected without re-installation.

In [1]:
# Installing RMB from Github Repository!

! git clone https://github.com/isri-aist/RoboManipBaselines.git --recursive
! cd RoboManipBaselines && pip install -e .

# For Colab
!apt-get update
!apt-get install -y ffmpeg
!pip uninstall -y torch torchvision torchaudio torchcodec
!pip install torch==2.6.0+cu124 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install torchcodec==0.2

Cloning into 'RoboManipBaselines'...
remote: Enumerating objects: 13076, done.
remote: Counting objects: 100% (2012/2012), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 13076 (delta 1932), reused 1835 (delta 1830), pack-reused 11064 (from 2)
Receiving objects: 100% (13076/13076), 91.25 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (8570/8570), done.
Updating files: 100% (898/898), done.
Submodule 'third_party/3D-Diffusion-Policy' (https://github.com/Naoki-Shibayama/3D-Diffusion-Policy.git) registered for path 'third_party/3D-Diffusion-Policy'
Submodule 'third_party/FlowPolicy' (https://github.com/mmurooka/FlowPolicy.git) registered for path 'third_party/FlowPolicy'
Submodule 'third_party/ManiFlow_Policy' (https://github.com/Naoki-Shibayama/ManiFlow_Policy.git) registered for path 'third_party/ManiFlow_Policy'
Submodule 'third_party/YCB_sim' (https://github.com/mmurooka/YCB_sim.git) registered for path 'third_party/YCB_sim'
Submodule 'third_party/act' (https:/

### Install Additional Dependencies for ACT

Some policies, like Action Chunking with Transformer (ACT), have additional dependencies. Here, we specifically install the extra packages required for ACT from the `RoboManipBaselines` root directory and then install `detr` within the ACT submodule's directory. This ensures that the ACT policy is fully functional.

In [2]:
# Installing Action Chunking with Transformer (ACT)
%cd RoboManipBaselines
! pip install -e .[act]

! cd third_party/act/detr && pip install -e .

Obtaining file:///content/RoboManipBaselines
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for robo_manip_baselines (pyproject.toml) ... done
  Created wheel for robo_manip_baselines: filename=robo_manip_baselines-2.0.0-0.editable-py3-none-any.whl size=8097 sha256=34b8089afeba16a1a34ecb706ac45c8ae9ef2cd735a8317f9bce7e3125a29bd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-56aydgkk/wheels/8c/91/46/a6d63aa4a9c72f97376acc3a87d7abaf51baea214009db1279
Successfully built robo_manip_baselines
  Attempting uninstall: robo_manip_baselines
    Found existing installation: robo_manip_baselines 2.0.0
    Uninstalling robo_manip_baselines-2.0.0:
      Successfully uninstalled robo_manip_baselines-2.0.0
Obtaining file:///content/RoboManipBaselines/third_party/act/detr
  Preparing metadata (setup.py) ... done
  Runni

## Download Sample Dataset

To demonstrate the training process, we'll download a sample dataset. The table below lists available datasets. For this tutorial, we will use the `MujocoUR5eCable_Dataset30` dataset, which contains 30 demonstrations for the `MujocoUR5eCable` environment. We download the dataset, unzip it, and then remove the zip file to save space.

### Download and Prepare `MujocoUR5eCable_Dataset30`

This step changes the current directory to the dataset directory within the cloned `RoboManipBaselines` repository. It then downloads the `MujocoUR5eCable_Dataset30.zip` file, creates a new directory for it, extracts the contents, and finally removes the downloaded zip file to keep the environment clean.

In [3]:
# Download the sample dataset (MujocoUR5eCable_Dataset30)
%cd /content/RoboManipBaselines/robo_manip_baselines/dataset
! wget -O MujocoUR5eCable_Dataset30.zip https://www.dropbox.com/scl/fo/sykc20cnax2scom1u8sc6/AM-zLM8dAZ5h6EQ8eDXcZic?rlkey=7icbmjc6wdqnp0tngfjqlhwoh&dl=1

! mkdir -p MujocoUR5eCable
! unzip MujocoUR5eCable_Dataset30.zip -d MujocoUR5eCable
! rm -r MujocoUR5eCable_Dataset30.zip

/content/RoboManipBaselines/robo_manip_baselines/dataset
--2025-11-17 02:30:24--  https://www.dropbox.com/scl/fo/sykc20cnax2scom1u8sc6/AM-zLM8dAZ5h6EQ8eDXcZic?rlkey=7icbmjc6wdqnp0tngfjqlhwoh
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3abdc9014b92991c626bf306c8.dl.dropboxusercontent.com/zip_download_get/CYuBBy1FuMN0OXeotZ_5QVUDsfzQEs9UTIQtctdIHn0sdb_AUiCqujki6c-RgcV-DQ7sYbqyJwJqsob0hAEO9cjtVcQdfSDNcS1Rsp7aWzLs_Q# [following]
--2025-11-17 02:30:26--  https://uc3abdc9014b92991c626bf306c8.dl.dropboxusercontent.com/zip_download_get/CYuBBy1FuMN0OXeotZ_5QVUDsfzQEs9UTIQtctdIHn0sdb_AUiCqujki6c-RgcV-DQ7sYbqyJwJqsob0hAEO9cjtVcQdfSDNcS1Rsp7aWzLs_Q
Resolving uc3abdc9014b92991c626bf306c8.dl.dropboxusercontent.com (uc3abdc9014b92991c626bf306c8.dl.dropboxusercontent.com)... 162.125.2.15, 2620:100:6017:15::a27d

## Run Training for ACT Policy

Finally, we'll run the training script for the ACT policy using the downloaded dataset. This involves navigating to the root of the `RoboManipBaselines` repository, and then executing the `Train.py` script with the `Act` policy and specifying the path to our downloaded dataset.

In [4]:
# Go to the top directory of this repository clone
%cd RoboManipBaselines/robo_manip_baselines
! python ./bin/Train.py Act --dataset_dir ./dataset/MujocoUR5eCable --batch_size 32

/content/RoboManipBaselines/robo_manip_baselines
2025-11-17 02:30:39.640285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763346639.893567    3348 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763346639.959594    3348 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763346640.472188    3348 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763346640.472226    3348 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763346640.472231    3348 